In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier  
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn import preprocessing
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
from numpy import asarray


pima_df = pd.read_csv('/kaggle/input/pima-indians-diabetes-database/diabetes.csv')
 
pima_df.describe()

def clean_data(df_raw,
               cols_with_zeros=['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age'],
               response = ['Outcome']):
    df = df_raw.copy()
    # replace zero with NaN in features
    df[cols_with_zeros] = df[cols_with_zeros].replace(0, np.nan)
    # remove \ and } from response
    df = df.replace(to_replace=r'\\|\}', value='', regex=True)
    # change response data type to int
    df[response] = df[response].astype('int')
    return df
 
pima_df_cleaned = clean_data(pima_df)
pima_df_cleaned.head()

values = pima_df_cleaned.values
imputer = SimpleImputer(missing_values=np.NaN, strategy='mean')

# transform the dataset
transformed_values = imputer.fit_transform(values)

transformed_values = pd.DataFrame(transformed_values)

pima_df_new = pd.DataFrame(transformed_values)

pima_df_new.isna().sum()

X = pima_df_new.iloc[:,0:8].values
Y = pima_df_new.iloc[:,8].values 

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3)


std_scale = preprocessing.StandardScaler().fit(x_train)
x_train_norm = std_scale.transform(x_train)
#Converting numpy array to dataframe
x_train = pd.DataFrame(x_train_norm)
print (x_train.head())

std_scale = preprocessing.StandardScaler().fit(x_test)
x_test_norm = std_scale.transform(x_test)
#Converting numpy array to dataframe
x_test = pd.DataFrame(x_test_norm)
print (x_test.head())

model = Sequential()
model.add(Dense(14, activation='relu', input_dim=x_train.shape[1]))
model.add(Dropout(0.5))
model.add(Dense(14, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

#sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='binary_crossentropy',
              optimizer= 'adam')

model.fit(x_train, y_train, epochs=50, batch_size=16)

#import xgboost as xgb
#xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
#                max_depth = 5, alpha = 10, n_estimators = 10)
#xg_reg.fit(x_train,y_train)

#preds = xg_reg.predict(x_test)

preds = model.predict(x_test)
preds[preds>=0.5] = 1
preds[preds<0.5] = 0
print( accuracy_score(y_test, preds))
